# Test to see if 2 summary prompts are equivalent

In [9]:
import requests
import json
import os

def sendchat(instructions):
    API_KEY = "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"
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    payload = {
        "model": "openai/gpt-4.1",
        "messages": [
            {
                "role": "user",
                "content": instructions,
            },
        ],
        "stream": False,
    }

    response = requests.post(url, headers=headers, json=payload)
    return response.json()

In [18]:
opinions = ["Current abortion laws are too laxed, it should be illegal in all states. no children should be killed.",
"I think everyone should bear arms freely. we should fight for our rights and protect ourselves. fuck the democrats",
"there should be more promotion opportunities for minorities and women in the workplace. Maternal leave should be longer and mandatory for all companies"]


In [21]:
responses = {}
for i, opinion in enumerate(opinions):
    prompt1 = f"""Create a one-sentence summary of the following argument.

    - Omit the reasons provided (e.g., "abortion should be legal because it is a right" → "abortion should be legal")
    - Avoid double-barreled questions (only one issue position and one concept at a time)
    - If multiple issues are mentioned, focus on the first
    - Stick to what the person says
    - Write it from the first person ("I believe ...")
    - Remove everything after "because"
    - Keep it under 15 words
    - Return only one statement (not "I believe X and Y", only "I believe X")

    Argument: {opinion}
    Summary:"""

    prompt2 = f"""Create a one-sentence summary of the following argument. The summary should start with "I believe" and only retain one clear position about one main issue from the original argument.

    Argument: {opinion}
    Summary:"""

    for n, prompt in {"prompt1": prompt1, "prompt2": prompt2}.items():
        response = sendchat(prompt)
        responses[f"{n}_{i}"] = response

In [22]:
responses['prompt1_1']

{'id': 'gen-1757007136-MM30bi11c2KbVZGX1RRm',
 'provider': 'OpenAI',
 'model': 'openai/gpt-4.1',
 'object': 'chat.completion',
 'created': 1757007136,
 'choices': [{'logprobs': None,
   'finish_reason': 'stop',
   'native_finish_reason': 'stop',
   'index': 0,
   'message': {'role': 'assistant',
    'content': 'I believe everyone should bear arms freely.',
    'refusal': None,
    'reasoning': None}}],
 'system_fingerprint': 'fp_3502f4eb73',
 'usage': {'prompt_tokens': 171,
  'completion_tokens': 8,
  'total_tokens': 179,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 0}}}

In [23]:
for n, response in responses.items():
    print(n, response["choices"][0]["message"])

prompt1_0 {'role': 'assistant', 'content': 'I believe abortion should be illegal in all states.', 'refusal': None, 'reasoning': None}
prompt2_0 {'role': 'assistant', 'content': 'I believe abortion should be illegal in all states.', 'refusal': None, 'reasoning': None}
prompt1_1 {'role': 'assistant', 'content': 'I believe everyone should bear arms freely.', 'refusal': None, 'reasoning': None}
prompt2_1 {'role': 'assistant', 'content': 'I believe everyone should have the right to bear arms.', 'refusal': None, 'reasoning': None}
prompt1_2 {'role': 'assistant', 'content': 'I believe there should be more promotion opportunities for minorities and women in the workplace.', 'refusal': None, 'reasoning': None}
prompt2_2 {'role': 'assistant', 'content': 'I believe there should be more promotion opportunities for minorities and women in the workplace.', 'refusal': None, 'reasoning': None}


## They seem to yield similar results -> change summary block.